UMAP 의 실습에 이용했던 fashion-MNIST 데이터를 이용합니다.

In [1]:
import sys
mnist_utils_path = '../../../../fashion-mnist/utils/'
mnist_data_path  = '../../../../fashion-mnist/data/fashion/'
sys.path.append(mnist_utils_path)

학습용 데이터와 테스트용 데이터를 모두 불러옵니다.

In [2]:
from mnist_reader import load_mnist

train, train_labels = load_mnist(mnist_data_path, kind='train')
test, test_labels   = load_mnist(mnist_data_path, kind='t10k')

print(train.shape)
print(train_labels.shape)
print(test.shape)
print(test_labels.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


IRIS + PCA tutorial 에서 Bokeh 를 이용하여 scatter plot 을 그리는 연습을 하였습니다. 이번에는 Bokeh 를 이용하여 여러 장의 이미지를 출력하는 연습을 해봅니다. figure 를 이용하여 도화지를 준비하는 과정은 동일합니다. 더하여 x, y 축의 레이블 및 grid lines 을 모두 보이지 않게 설정하였습니다.

image 함수는 그림을 출력하는 함수로, image(x, y) 는 이미지의 좌측 하단에 위치하는 값, dw 는 x 축의 범위, dh 는 y 축의 범위입니다. 한 장의 이미지만을 출력할 때에는 (0, 1) 로 설정하여도 됩니다. 이 값은 이후에 scatter plot 과 image 를 동시에 출력할 때 잘 조절해야 합니다.

gridplot 함수는 list of list (rows of columns) 로 이뤄진 그림들을 입력값으로 받습니다. 그리고 show 함수를 이용하여 이 gridplot 을 출력합니다.

In [3]:
import math
from bokeh.plotting import output_notebook, show, figure
from bokeh.layouts import gridplot

output_notebook()

def show_image_grid(images, labels, n_cols=4):
    figs = []
    for img, l in zip(images, labels):
        p = figure(width=150, height=150, title=f'Label {l}')
        p.axis.visible = False
        p.xgrid.visible = False
        p.ygrid.visible = False
        p.image(image=[img], x=0, dw=1, y=0, dh=1)
        figs.append(p)

    # 소수점 올림
    n_rows = math.ceil(len(figs) / n_cols)
    rows = []
    for i in range(n_rows):
        b = i * n_cols
        e = (i+1) * n_cols
        rows.append(figs[b:e])

    # input = list of list
    gp = gridplot(rows)
    show(gp)

    return rows

Loading BokehJS ...

Image handling tutorial 에서 연습한 것처럼 flatten vector 를 reshape 함수를 이용하여 이미지로 복원합니다. 행렬의 (0, 0) 은 이미지의 우측 상단이기 때문에 90 도 방향으로 2 번 회전하였습니다.

In [4]:
import numpy as np

image_samples = [np.rot90(xi.reshape(28,28),k=2) for xi in train[:12]]
label_samples = train_labels[:12]

앞서 만든 함수를 이용하여 이미지를 확인해 봅니다.

In [5]:
rows = show_image_grid(image_samples, label_samples)

Scikit-learn 의 multi-layer perceptron classifier 를 이용하여 이를 분류하는 모델을 학습해 봅니다. 학습 및 성능 평가를 반복할 것이기 때문에 이 과정 역시 함수로 만듭니다. Python 의 time package 를 이용하면 현재 시각을 UNIX time (1970.01.01 00:00:00 으로부터의 seconds) 으로 표현할 수 있습니다. 이 시간의 차이를 계산하여 학습 및 테스트 시간을 측정합니다. 그리고 classification_report 함수를 이용하여 클래스 별 성능을 확인합니다.

In [6]:
from time import time
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn import metrics


def train_and_evaluate(model, scaler, X_train, y_train, X_test, y_test):
    # training
    t = time()
    X_train_scaled = scaler.fit_transform(X_train)
    model.fit(X_train_scaled, y_train)
    t = time() - t
    print(f'training time = {t:.3} secs. with n={X_train.shape[0]}')

    # test
    t = time()
    X_test_scaled = scaler.transform(X_test)
    y_test_pred = model.predict(X_test_scaled)
    t = time() - t
    print(f'test     time = {t:.3} secs. with n={X_test.shape[0]}\n')
    print(metrics.classification_report(y_test, y_test_pred))

    return model

학습데이터는 클래스의 순서가 뒤섞여 있는 데이터입니다.

In [7]:
train_labels[:10]

array([9, 0, 0, 3, 0, 2, 7, 2, 5, 5], dtype=uint8)

우선 3000 개의 데이터만 이용하여 모델을 학습합니다. hidden layer size, activation function, optimizer 등을 설정합니다. `warm_start = True` 로 설정하면 모델이 충분히 학습되지 않았을 때 현재 weights 를 초기값으로 추가학습을 할 수 있습니다.

In [8]:
model = MLPClassifier(
    hidden_layer_sizes = (200, 50),
    activation = 'relu',
#     solver = 'sgd',
    solver = 'adam',
    warm_start = True,
    max_iter = 1000,
    epsilon = 0.00001
)
scaler = StandardScaler()

n_samples = 3000
X = train[:n_samples]
y = train_labels[:n_samples]

model = train_and_evaluate(model, scaler, X, y, test, test_labels)

training time = 6.56 secs. with n=3000
test     time = 0.0989 secs. with n=10000

              precision    recall  f1-score   support

           0       0.80      0.80      0.80      1000
           1       0.97      0.95      0.96      1000
           2       0.70      0.73      0.72      1000
           3       0.82      0.85      0.84      1000
           4       0.73      0.77      0.75      1000
           5       0.93      0.87      0.90      1000
           6       0.62      0.57      0.59      1000
           7       0.87      0.92      0.90      1000
           8       0.95      0.92      0.93      1000
           9       0.92      0.93      0.92      1000

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



성능이 아쉽습니다. 이번에는 학습데이터를 20000 개로 늘려서 추가학습을 합니다. 그런데 데이터의 개수가 증가하였음에도 불구하고 학습 시간이 매우 짧습니다. 이는 모델이 이미 weights 가 수렴했다고 판단했기 때문입니다.

In [9]:
n_samples = 20000
X = train[:n_samples]
y = train_labels[:n_samples]

model = train_and_evaluate(model, scaler, X, y, test, test_labels)

training time = 0.884 secs. with n=20000
test     time = 0.0972 secs. with n=10000

              precision    recall  f1-score   support

           0       0.80      0.80      0.80      1000
           1       0.99      0.95      0.97      1000
           2       0.68      0.83      0.74      1000
           3       0.79      0.90      0.84      1000
           4       0.83      0.63      0.71      1000
           5       0.93      0.89      0.91      1000
           6       0.62      0.58      0.60      1000
           7       0.92      0.88      0.90      1000
           8       0.93      0.95      0.94      1000
           9       0.90      0.94      0.92      1000

    accuracy                           0.83     10000
   macro avg       0.84      0.83      0.83     10000
weighted avg       0.84      0.83      0.83     10000



만약 `warm_start=False` 로 설정하면 re-initialize 되어 모델이 다시 학습됩니다. 성능이 조금이지만 개선되었습니다. 아마도 처음 3000 개의 데이터로 학습한 모델은 성능이 좋지 않은 local optima 에 갇혀있는 것이라 예상됩니다. 하지만 여전히 몇몇 클래스에 대해서는 성능이 아쉽습니다. 모델의 구조 (hidden layer size) 를 변경하던지 혹은 image classification 에 좋다고 알려진 convolutional neural network 를 이용해 보아야 할 것 같습니다.

In [10]:
n_samples = 20000
X = train[:n_samples]
y = train_labels[:n_samples]

model.warm_start = False
model = train_and_evaluate(model, scaler, X, y, test, test_labels)

training time = 39.1 secs. with n=20000
test     time = 0.0974 secs. with n=10000

              precision    recall  f1-score   support

           0       0.82      0.82      0.82      1000
           1       0.98      0.96      0.97      1000
           2       0.78      0.81      0.80      1000
           3       0.87      0.87      0.87      1000
           4       0.79      0.80      0.79      1000
           5       0.95      0.94      0.95      1000
           6       0.71      0.69      0.70      1000
           7       0.94      0.96      0.95      1000
           8       0.96      0.96      0.96      1000
           9       0.95      0.95      0.95      1000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



Feed forward neural network 의 패러매터들을 살펴봅니다. 이를 위하여 여러 개의 iterable 한 객체 내 데이터를 살펴보는 zip 함수를 이용합니다. zip 함수에 두 개 이상의 list, set 등을 입력하면 길이가 짧은 객체 탐색이 끝날 때까지 각 객체 내 데이터를 함께 yield 합니다.

In [11]:
# Python zip
l1 = list('abcde')
l2 = list(range(3,6))

print(f'l1 = {l1}')
print(f'l2 = {l2}\n')

for item1, item2 in zip(l1, l2):
    print(f'item1 = {item1}, item2 = {item2}')

l1 = ['a', 'b', 'c', 'd', 'e']
l2 = [3, 4, 5]

item1 = a, item2 = 3
item1 = b, item2 = 4
item1 = c, item2 = 5


enumerate 함수는 데이터와 해당 데이터의 index 를 함께 출력합니다.

In [12]:
# Python enumerate
for i, l1 in enumerate(l1):
    print(f'#{i} = {l1}')

#0 = a
#1 = b
#2 = c
#3 = d
#4 = e


이를 이용하여 모델의 패러매터들의 크기를 살펴봅니다. 이 모델은 784 -> 200 -> 50 으로 hidden representation 을 변경한 뒤, 10 개의 클래스로 구분하는 softmax regression 을 수행합니다. 이에 해당하는 weights 와 intercept 가 다음의 크기로 학습되었습니다.

In [13]:
print(f'num layers = {len(model.coefs_)}')

coefs = model.coefs_
intercepts = model.intercepts_

for l, (coef, intercept) in enumerate(zip(coefs, intercepts)):
    print(f'layer#{l+1}: coef={coef.shape}, intercept={intercept.shape}')

num layers = 3
layer#1: coef=(784, 200), intercept=(200,)
layer#2: coef=(200, 50), intercept=(50,)
layer#3: coef=(50, 10), intercept=(10,)


In [14]:
coefs[0][:3,:3]

array([[ 0.00906187,  0.04710608,  0.02653178],
       [-0.0517881 , -0.04819394, -0.01113287],
       [ 0.06830596, -0.0948201 , -0.0511848 ]])